In [0]:
# !export IMAGE_FAMILY="pytorch-latest-gpu"
# !export ZONE="us-west1-b"
# !export INSTANCE_NAME="pytorch-colab-backend"
# !gcloud compute ssh --zone ZONE INSTANCE_NAME -- -L 8888:localhost:8888

In [1]:
!pip install transformers==2.5.0

In [2]:
import numpy as np
import pandas as pd
import transformers
from transformers import BertModel, AdamW, BertConfig, BertTokenizer, Model2Model, PreTrainedEncoderDecoder, BertPreTrainedModel, AutoTokenizer, AutoModelWithLMHead
from torch.utils.data import Dataset
import torch
import re
from torch import nn
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [0]:
#HRbot w/o end and start answer tokens. Answer parts are located in different context sentences

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
data = pd.read_csv('drive/My Drive/hrbot.csv')
data.head()

,Question,Answer
0,Какое управленческое действие относится к функ...,"планирование, прогнозирование, мотивация, орга..."
1,Управленческий персонал включает:,"руководителей, специалистов"
2,К функциям менеджмента относят,"планирование, прогнозирование, мотивация, орга..."
3,К японскому менеджменту персонала относится:,продвижение зависит от возраста рабочего или ...
4,С какими дисциплинами связана система наук о т...,"экономика труда, психология труда, физиология ..."


Идея - берем hidden state cls берта вопросов, берем hidden state cls берта контекста (учебник), по косинусному расстоянию ищем топ ближайших предложений контекста, для слов из данных предложений берем эмбеддинги с последнего слоя берта, делаем механизм влияния на вопрос, шлем в транформер декодер и решаем задачу абстрактной суммаризации

In [0]:

config_path = '/content/drive/My Drive/Rubert/rubert_cased_L-12_H-768_A-12_pt/bert_config.json'
checkpoint_path = '/content/drive/My Drive/Rubert/rubert_cased_L-12_H-768_A-12_pt/pytorch_model.bin'
vocab_path = '/content/drive/My Drive/Rubert/rubert_cased_L-12_H-768_A-12_pt/vocab.txt'
#data tokenizetion for bert
class BotDataset(Dataset):
  def __init__(self, data, column, context_atn=False,max_leninf =False):
    self.df = data[column].copy()
    # self.tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
    self.tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=True)
    # self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
    if max_leninf == False:
      self.maxlen =  max(data[column].apply(lambda x:len(x)))
    else:
      self.maxlen = max_leninf
    self.context_atn = context_atn

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    sentence = self.df.loc[index]
    tokens = self.tokenizer.tokenize(sentence)
    # if self.context_atn == False:
      # tokens = ['[CLS]'] + tokens + ['[SEP]']
    if len(tokens) < self.maxlen:
      tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))]
    else:
      if self.context_atn == False:
        tokens = tokens[:(self.maxlen-1)] + ['[SEP]']
      else: 
        tokens = tokens[:self.maxlen]
    tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
    tokens_ids_tensor = torch.tensor(tokens_ids)
    attn_mask = (tokens_ids_tensor != 0).long()
    return tokens_ids_tensor, attn_mask

In [0]:
data_len = len(data)

In [7]:
border = int(0.8*data_len)
Question_dataset_train = BotDataset(data.iloc[:border], 'Question')
Question_dataset_test = BotDataset(data.iloc[border:], 'Question')
Answer_dataset_train = BotDataset(data.iloc[:border], 'Answer', False, 16)
Answer_dataset_test = BotDataset(data.iloc[border:], 'Answer', False, 16)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [8]:
#Context = hr management issues database
context = open('drive/My Drive/context.txt', 'r').read()
print(len(context))
print(context[:30])
pattern = re.compile('[^А-Яа-яЁёA-Za-z0-9/./,/:/?/]')
context = pattern.sub(' ', context)
context = pd.DataFrame(context.split('.'), columns = ['sentences'])
context_len = len(context)


796689
﻿УПРАВЛЕНИЕ ПЕРСОНАЛОМ
Под ред


In [9]:
# context for choosing appropriate sentences for attention 
context_dataset = BotDataset(context, 'sentences', False,16)

# context for answer generation
context_dataset_answgen = BotDataset(context, 'sentences', True,16)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
#get CLS embedding 
# model_qc = BertModel.from_pretrained('bert-base-multilingual-uncased').to(device)
model_qc = BertModel.from_pretrained(checkpoint_path, config=config_path).to(device)
# model_qc = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence").cuda()
model_qc.eval()

#question
q_tensor_train = torch.stack([Question_dataset_train[x][0] for x in range(border)]).to(device)
q_tensor_test = torch.stack([Question_dataset_test[x][0] for x in range(border, data_len)]).to(device)
q_atn_train = torch.stack([Question_dataset_train[x][1] for x in range(border)]).to(device)
q_atn_test = torch.stack([Question_dataset_test[x][1] for x in range(border, data_len)]).to(device)
with torch.no_grad():
  q_hid_train, _ = model_qc(q_tensor_train, q_atn_train)
  q_hid_test, _ = model_qc(q_tensor_test, q_atn_test)
q_hid_train = q_hid_train[:, 0]
q_hid_test = q_hid_test[:, 0]


In [0]:
a_tensor_train = torch.stack([Answer_dataset_train[x][0] for x in range(border)]).to(device)
a_tensor_test = torch.stack([Answer_dataset_test[x][0] for x in range(border, data_len)]).to(device)

a_tensor_atn_train = torch.stack([Answer_dataset_train[x][1] for x in range(border)]).to(device)
a_tensor_atn_test = torch.stack([Answer_dataset_test[x][1] for x in range(border, data_len)]).to(device)

with torch.no_grad():
  a_tensor_train_embed, _ = model_qc(a_tensor_train , a_tensor_atn_train)
  # a_tensor_test_embed, _ = model_qc(a_tensor_test , a_tensor_atn_test)

In [0]:
train_dataset = torch.utils.data.TensorDataset(q_hid_train, a_tensor_train, a_tensor_atn_train, a_tensor_train_embed)
test_dataset = torch.utils.data.TensorDataset(q_hid_test, a_tensor_test, a_tensor_atn_test)

In [0]:
#context for choosing sentence
c_tensor = torch.stack([context_dataset[x][0] for x in range(context_len)]).to(device)
c_atn = torch.stack([context_dataset[x][1] for x in range(context_len)]).to(device)
with torch.no_grad():
  c_hid, _ = model_qc(c_tensor, c_atn)
  c_hid = c_hid[:,0]

In [0]:
#context for text generation
c_tensor = torch.stack([context_dataset_answgen[x][0] for x in range(context_len)]).to(device)
c_atn = torch.stack([context_dataset_answgen[x][1] for x in range(context_len)]).to(device)
with torch.no_grad():
  w_embed, _ = model_qc(c_tensor, c_atn)

In [17]:
c_tensor[0]

tensor([14524, 75212,  2068,  9633,  1827, 15034,   867,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0], device='cuda:0')

In [18]:
q_hid_train.size()

torch.Size([544, 768])

In [19]:
w_embed.size()

torch.Size([6009, 32, 768])

In [0]:
#settings
batch_size = 12
embedding_dim = q_hid_train.size()[1]
attn_size = 16
learning_rate = 0.0005

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False, drop_last=True)

In [17]:
# tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
tokenizer.sep_token_id

102

In [0]:
# Source: https://github.com/huggingface/transformers/pull/1455/files  p.s. WIP code, so changes are:  mistakes fixed, prob list added, model changed
softm = torch.nn.Softmax(dim=1)
class TransformerBeamSearch(nn.Module):
    def __init__(
        self,
        model,
        fc,
        vocab_size,
        tokenizer,
        batch_size,
        beam_size,
        min_length,
        max_length,
        alpha=0,
        block_repeating_trigram=True,
    ):
        """
        Attributes:
            mask_word_id: token id that corresponds to the mask
        """
        super(TransformerBeamSearch, self).__init__()
        self.model = model
        self.fc = fc
        # decoder_config = transformers.AutoConfig.from_pretrained('bert-base-multilingual-uncased', is_decoder=True)
        self.end_token_id = tokenizer.sep_token_id
        self.start_token_id = tokenizer.cls_token_id
        self.beam_size = beam_size
        self.min_length = min_length
        self.max_length = max_length
        self.batch_size = batch_size
        self.vocab_size = vocab_size

        self.block_repeating_trigram = block_repeating_trigram
        self.apply_length_penalty = False if alpha == 0 else True
        self.alpha = alpha

        # State of the beam
        self.hypotheses = [[] for _ in range(batch_size)]
        self.batch_offset = torch.arange(batch_size, dtype=torch.long).cuda()
        self.beam_offset = torch.arange(
            0, batch_size * self.beam_size, step=self.beam_size, dtype=torch.long
        ).cuda()
        self.growing_beam = torch.full(
            (batch_size * self.beam_size, 1), self.start_token_id, dtype=torch.long
        )
        self.growing_prob = torch.full(
            (batch_size,1,self.vocab_size), 0, dtype=torch.long
        ).cuda()

        self.topk_log_probabilities = torch.tensor(
            [0.0] + [float("-inf")] * (self.beam_size - 1), dtype=torch.float
        ).repeat(batch_size)
        self.results = {
            "predictions": [[] for _ in range(batch_size)],
            "scores": [[] for _ in range(batch_size)],
            "probs": [[] for _ in range(batch_size)],
        }
        self._step = 0
        self.is_done = False

    def _generate_square_subsequent_mask(self, sz):
      mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
      mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
      return mask 

    def _generate_subsequent_mask(self, t, s):
      mask = (torch.triu(torch.ones(s, t)) == 1).transpose(0, 1)
      mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
      return mask      

    def step(self, log_probabilities):
        """ Grows the beam by one step. """
        self._step += 1

        # The batch size changes as some beams finish so we define _B
        vocab_size = log_probabilities.size(-1)
        _B = log_probabilities.size(0) // self.beam_size

        # Multiply each beam probability with the probability of the
        # next token (conditioned on the words in the beam).
      

        log_probabilities = log_probabilities.squeeze(1)
        
        log_probabilities += self.topk_log_probabilities.view(-1, 1).cuda()

        log_probabilities = self.enforce_min_length(log_probabilities)
        if self.block_repeating_trigram:
            self.remove_repeating_trigrams(log_probabilities, _B)

       
        # Find the `beam_size` (previous_beam + token) combinations with
        # the highest score
        topk_log_probabilities, topk_ids = torch.topk(
            log_probabilities.view(_B, self.beam_size * vocab_size),
            self.beam_size,
            dim=1) 
        prob = log_probabilities.view(_B, -1, vocab_size)
        prob = torch.max(prob, dim=1).values.unsqueeze(1).long()
        prob = (prob >= torch.min(topk_log_probabilities, dim=1).values.unsqueeze(1))*1
        
        
       

        # Apply the length penalty. The +1 accounts for the [EOS] token
        # that will be added if the beam ends.
        topk_scores = topk_log_probabilities / self.length_penalty()
        
        # Retrieve the corresponding respective beam and token id
        # topk_token_ids[i] will be added to topk_beam_ids[i]
        topk_beam_ids = topk_ids.div(vocab_size)
        topk_token_ids = topk_ids.fmod(vocab_size)
        
       
        # Retrieve the row index of the surviving beams in the original
        # view of the log_probabilities tensor
        surviving_beams_rows = (topk_beam_ids + self.beam_offset[:_B].view(-1, 1).cuda()).view(
            -1
        )

        # Append the last predictions
        self.growing_beam = torch.cat(
            [
                self.growing_beam.index_select(0, surviving_beams_rows),
                topk_token_ids.view(-1, 1),
            ],
            1,
        )
        self.growing_prob = torch.cat(
            [
                self.growing_prob,
                prob.cuda(),
            ],
            1,
        )


        # Check if any of the beam searches has ended during this
        # growth step. Also if top beam (most probable) has ended
        # for one element of the batch.
        
        is_finished = topk_token_ids.eq(self.end_token_id)
       
        is_finished = self.enforce_max_length(is_finished)
        
        is_top_beam_finished = is_finished[:, 0].eq(1)

        # Save the finished searches
        
        if is_finished.any():
            predictions = self.growing_beam.view(-1, self.beam_size, self.growing_beam.size(1))
            for i in range(is_finished.size(0)):
                if is_top_beam_finished[i]:
                    is_finished[i].fill_(1)
                finished_hyp = is_finished[i].nonzero().view(-1)

                # Store finished hypotheses for this batch.
                b = self.batch_offset[i]
                for j in finished_hyp:
                    self.hypotheses[b].append((topk_scores[i, j], predictions[i, j, :]))

                # If the batch reached the end, save the best hypotheses
                # in terms of length-penalized score.
                if is_top_beam_finished[i]:
                    best_hyp = sorted(
                        self.hypotheses[b], key=lambda x: x[0], reverse=True
                    )
                    best_score, best_prediction = best_hyp[0]
                    self.results["scores"][b].append(best_score)
                    self.results["predictions"][b].append(best_prediction)
                    self.results["probs"][b].append(self.growing_prob[b])

            non_finished = is_top_beam_finished.eq(0).nonzero().view(-1).cuda()
            if len(non_finished) == 0:
                self.is_done = True

            # Remove finished batches for the next step.
            topk_log_probabilities = topk_log_probabilities.index_select(
                0, non_finished
            )
            self.batch_offset = self.batch_offset.index_select(0, non_finished)
            self.growing_beam = predictions.index_select(0, non_finished).view(
                -1, self.growing_beam.size(-1)
            )

            surviving_beams_rows = surviving_beams_rows.index_select(0, non_finished)

        return surviving_beams_rows

    def forward(self, encoder_outputs,encoder_embeddings):
        # keyword arguments come in 3 flavors: encoder-specific (prefixed by
        # `encoder_`), decoder-specific (prefixed by `decoder_`) and those
        # that apply to the model as whole.
        # We let the specific kwargs override the common ones in case of conflict.
        # kwargs_encoder = {
        #     argument[len("encoder_"):]: value
        #     for argument, value in kwargs.items()
        #     if argument.startswith("encoder_")
        # }
        # kwargs_decoder = {
        #     argument[len("decoder_"):]: value
        #     for argument, value in kwargs.items()
        #     if argument.startswith("decoder_")
        # }
        # kwargs_common = {
        #     argument: value
        #     for argument, value in kwargs.items()
        #     if not (argument.startswith("encoder_") or argument.startswith("decoder_"))
        # }
        # kwargs_decoder = dict(kwargs_common, **kwargs_decoder)
        # kwargs_encoder = dict(kwargs_common, **kwargs_encoder)

        # forward pass on the encoder
  
        encoder_embeddings = encoder_embeddings.permute(1,0,2)
        encoder_hidden_states = tile(
            encoder_embeddings, self.beam_size, dim=0, variants=1,
        )
        encoder_outputs = tile(
            encoder_outputs, self.beam_size, dim=0, variants=2,
        )

        
        
        # grow the beam by generating sequences in an autoregressive way
        self.growing_beam = torch.full(
            (self.batch_size * self.beam_size, 1), self.start_token_id, dtype=torch.long).cuda()
        for step in range(self.max_length):
            decoder_input = self.growing_beam
            # [:, -1]
            # decoder_input = decoder_input.view(-1,1)
            decoder_input_atn = ((decoder_input == 0)*1).cuda()
            
            attn_mask = ((encoder_outputs == 0)*1).cuda()
           
           
            with torch.no_grad():
               decoder_input, _ = model_qc(decoder_input , decoder_input_atn)
            decoder_input_ch = decoder_input.permute(1,0,2)
            encoder_hidden_states = encoder_hidden_states.permute(1,0,2)
          
            mask = self._generate_subsequent_mask(decoder_input_ch.size(0),encoder_hidden_states.size(0)).to(device)
            mask_targ = self._generate_square_subsequent_mask(len(decoder_input_ch)).to(device)
            
            outputs = self.model(decoder_input_ch.cuda(), encoder_hidden_states.cuda(), mask_targ ,mask.cuda(),decoder_input_atn.bool(), attn_mask.bool() )
            outputs = outputs[-1].unsqueeze(0)
            
            outputs = outputs.permute(1,0,2)
            outputs = self.fc(outputs)
            
            
            log_probabilities = torch.nn.functional.log_softmax(outputs)
           
            surviving_beams_rows = self.step(log_probabilities)
            if self.is_done:
                break
            encoder_hidden_states = encoder_hidden_states.permute(1,0,2)
            encoder_hidden_states = encoder_hidden_states.index_select(0, surviving_beams_rows)
            encoder_outputs = encoder_outputs.index_select(0, surviving_beams_rows)

        return self.results

    def remove_repeating_trigrams(self, log_probabilities, _B):
        if(self._step + 1 > 1):
            for i in range(_B * self.beam_size):
                tokens = [t for t in self.growing_beam[i]]
                if(self._step + 1 > 3):
                  trigrams = [(tokens[k-1], tokens[k], tokens[k+1]) for k in range(1, len(tokens) - 1)]
                  last_trigram = tuple(trigrams[-1])
                  if last_trigram in trigrams[:-1]:
                      log_probabilities[i] = -1e20
        
                if tokens[-1] in tokens[:-1]:
                    log_probabilities[i] = -1e20
                

    def enforce_min_length(self, log_probabilities):
        if self._step < self.min_length:
            log_probabilities[:, self.end_token_id] = -1e20
        return log_probabilities

    def enforce_max_length(self, is_finished):
        if self._step + 1 == self.max_length:
            is_finished.fill_(1)
        return is_finished

    def length_penalty(self):
        return ((5.0 + (self._step + 1)) / 6.0) ** self.alpha


def tile(x, count, dim=0, variants=1):
    """
    Tiles `x` along dimension `dim` `count` times.
    Example:
        >> ex = torch.tensor([1,2],[3,4])
        >> tile(ex, 2, 0)
        torch.Tensor([[1,2],[1,2],[3,4],[3,4]])
    """
    perm = list(range(len(x.size())))
    if dim != 0:
        if variants == 1:
          perm[0], perm[dim], perm[2] = perm[dim], perm[0], perm[2]
        else:
          perm[0], perm[dim] = perm[dim], perm[0]
        x = x.permute(perm).contiguous()
    out_size = list(x.size())
    out_size[0] *= count
    batch = x.size(0)
    if variants == 1:
      x = (
          x.view(batch, -1, 768)
          .transpose(0, 1)
          .repeat(count, 1, 1)
          .transpose(0, 1)
          .contiguous()
          .view(*out_size)
      )
    else:
      x = (
          x.view(batch, -1)
          .transpose(0, 1)
          .repeat(count, 1)
          .transpose(0, 1)
          .contiguous()
          .view(*out_size)
      )      
    if dim != 0:
        x = x.permute(perm).contiguous()
    return x



In [0]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
class HRbotAnswerGen(nn.Module):
  def __init__(self):
    super(HRbotAnswerGen, self).__init__()
    # self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
    self.tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=True)
    # self.tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
    # decoder_config = transformers.AutoConfig.from_pretrained('bert-base-multilingual-uncased', is_decoder=True)
    # self.model_gen = Model2Model.from_pretrained('bert-base-multilingual-uncased', decoder_config=decoder_config)
    self.Q = nn.Linear(embedding_dim, attn_size)
    torch.nn.init.xavier_uniform(self.Q.weight)
    self.K = nn.Linear(embedding_dim, attn_size)
    torch.nn.init.xavier_uniform(self.K.weight)
    self.V = nn.Linear(embedding_dim, embedding_dim)
    torch.nn.init.xavier_uniform(self.V.weight)
    self.decoder_layer = nn.TransformerDecoderLayer(d_model=768, nhead=12)
    self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=8)
    # self.embed = nn.Embedding(self.tokenizer.vocab_size,embedding_dim)
    self.fc = nn.Linear(embedding_dim, self.tokenizer.vocab_size)
    torch.nn.init.xavier_uniform(self.fc.weight)

  def _generate_square_subsequent_mask(self, sz):
      mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
      mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
      return mask 

  def _generate_subsequent_mask(self, t, s):
      mask = (torch.triu(torch.ones(s, t)) == 1).transpose(0, 1)
      mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
      return mask  
    
  def forward(self, w_embed, q_hid_batch, c_hid, c_tensor, c_atn, a_tensor = None, a_tensor_atn = None, test = False, a_tensor_embed = None):
    
    q = 0
    for question in q_hid_batch:
      cos_q_tensor = question.repeat(c_hid.size(0),1)
      cos_qc_tensor = cos(cos_q_tensor, c_hid)
      _, attn_index = torch.sort(cos_qc_tensor, descending=True)
      attn_index = attn_index[:10]
      for index in attn_index:
        w_embed_at = w_embed[index]
        c_tensor_at = c_tensor[index]
        # c_atn_at = c_atn[index]
        if attn_index[0] == index:
          w_embed_at_total = w_embed_at.clone()
          c_tensor_at_total = c_tensor_at.clone()
          # c_atn_at_total = c_atn_at.clone()
        else:
          w_embed_at_total = torch.cat([w_embed_at_total, w_embed_at], dim=0)
          c_tensor_at_total = torch.cat([c_tensor_at_total, c_tensor_at], dim=0)
          # c_atn_at_total = torch.cat([c_atn_at_total, c_atn_at], dim=0)
      if q == 0:
        question_total = w_embed_at_total.unsqueeze(0).clone()
        question_context = c_tensor_at_total.unsqueeze(0).clone()
        # question_c_atn = c_atn_at_total.unsqueeze(0).clone()
        q = q + 1
      else:
        question_total = torch.cat([question_total, w_embed_at_total.unsqueeze(0)], dim=0)
        question_context = torch.cat([question_context, c_tensor_at_total.unsqueeze(0)], dim=0)
        # question_c_atn = torch.cat([question_c_atn, c_atn_at_total.unsqueeze(0)], dim=0)    

    q_hid_batch = q_hid_batch.unsqueeze(1)
    Q_question = self.Q(q_hid_batch)
    K_wembed = self.K(question_total)
    V_embedgen = self.V(question_total)
    softm_val = softm(torch.matmul(Q_question, K_wembed.permute(0,2,1))/(attn_size**(1/2)))
    softm_val = softm_val.squeeze(1)


    # --------for embedding input--------------------------------
    softm_matrix = torch.zeros(question_total.size(1),question_total.size(1)).repeat(batch_size,1,1).to(device)
    for i, _ in enumerate(softm_matrix):
      softm_matrix[i][torch.eye(question_total.size(1)).byte()] =softm_val[i]
    question_context_embed = torch.matmul(softm_matrix, V_embedgen)
    


    #----------for index input----------------------------------
    # softm_val = softm_val/torch.max(softm_val, dim=1).values.unsqueeze(1)
    # softm_val = torch.round(softm_val)
    # question_context = softm_val * question_context
   
    # self.model_gen.encoder.requires_grad_=False
    # self.model_gen.decoder.requires_grad_=False
    # self.model_gen.decoder.cls.requires_grad_=True

    
    del cos_q_tensor
    del cos_qc_tensor
    del w_embed_at_total
    del c_tensor_at_total
    # del c_atn_at_total
    del w_embed_at
    del c_tensor_at
    # del c_atn_at  
    del question_total
    # del question_c_atn
    
    question_context_embed = question_context_embed.permute(1,0,2)
    # question_context = question_context.permute(1,0)
    if a_tensor_embed != None:
      a_tensor_embed = a_tensor_embed.permute(1,0,2)
      a_tensor_atn = ((a_tensor_atn == 0)*1).to(device)
    # a_tensor_atn = a_tensor_atn.permute(1,0)

    question_context_atn = ((question_context == 0)*1).to(device)
    torch.cuda.empty_cache()
   

    if test == False:
      mask = self._generate_subsequent_mask(a_tensor_embed.size(0),question_context_embed.size(0)).to(device)
      mask_targ = self._generate_square_subsequent_mask(a_tensor_embed.size(0)).to(device)
      # a_tensor = self.embed(a_tensor)
      output = self.transformer_decoder(a_tensor_embed,question_context_embed,mask_targ ,mask,a_tensor_atn.bool(),question_context_atn.bool())
      output = output.permute(1,0,2)
      output = self.fc(output)
      
      # model_kwargs = {"encoder_attention_mask": question_context_atn, "decoder_attention_mask": a_tensor_atn }
      # out, _ , _= self.model_gen(encoder_input_ids=question_context.long(),decoder_input_ids=a_tensor.long(), **model_kwargs)
      # model.encoder.layer[3].attention.self.key.weight.reqieres_grad = True
      return output
    else:
      # model_kwargs = {"encoder_attention_mask": question_context_atn}
      tran_beams = TransformerBeamSearch(self.transformer_decoder, self.fc,self.tokenizer.vocab_size,self.tokenizer, batch_size = q_hid_batch.size(0),beam_size=5, min_length = 16 ,max_length = 16)
      
      out = tran_beams.forward(question_context, question_context_embed)
      score = torch.stack([x[0] for x in out['scores']])
      
      prob = torch.stack([x[0] for x in out['probs']])
   
      pred = torch.stack([x[0] for x in out['predictions']])
      return score, pred, prob

In [0]:
# tgt – the sequence to the decoder (required).

# memory – the sequnce from the last layer of the encoder (required).

# tgt_mask – the mask for the tgt sequence (optional).

# memory_mask – the mask for the memory sequence (optional).

# tgt_key_padding_mask – the mask for the tgt keys per batch (optional).

# memory_key_padding_mask – the mask for the memory keys per batch (optional).

In [26]:
model = HRbotAnswerGen()

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
epochs = 20

In [0]:
def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    total_loss = 0
  
    for i in range(len(real[0])):
        
      mask = real[:,i].ge(1).type(torch.cuda.FloatTensor)
      crit = nn.CrossEntropyLoss()
      loss_ = crit(pred[:,i].type(torch.cuda.FloatTensor), real[:,i].long()) * mask 
      total_loss  =  total_loss + torch.mean(loss_)
    return total_loss/len(real[0])

In [0]:

def evaluate(model, test_loader, device, c_hid, c_tensor, c_atn, w_embed):
  model.eval()

  for i, (q_hid_batch_test, answer_batch_test, answer_atn_batch_test) in enumerate(test_loader):
    q_hid_batch_test, answer_batch_test, answer_atn_batch_test = q_hid_batch_test.to(device), answer_batch_test.to(device), answer_atn_batch_test.to(device)
    
    with torch.no_grad():

      test_output = model(w_embed, q_hid_batch_test, c_hid, c_tensor, c_atn, test=True)
    
    #use scores as logits
    
    test_loss = loss_function(answer_batch_test, test_output[2])
    return test_loss

In [0]:
def train(model, train_loader, test_loader, device, epochs,c_hid, c_tensor, c_atn, w_embed):
  model.to(device)
  model.train()
  c_hid, c_tensor, c_atn, w_embed = c_hid.to(device), c_tensor.to(device), c_atn.to(device), w_embed.to(device)
  optimizer = AdamW(model.parameters(), lr=learning_rate)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
  for epoch in range(epochs):
    
    total_loss = 0
    for i , (q_hid_batch, answer_batch, answer_atn_batch, a_tensor_train_embed) in enumerate(train_loader):
      optimizer.zero_grad()
      
      q_hid_batch, answer_batch, answer_atn_batch, a_tensor_train_embed = q_hid_batch.to(device), answer_batch.to(device), answer_atn_batch.to(device), a_tensor_train_embed.to(device)

      logits = model(w_embed, q_hid_batch, c_hid, c_tensor, c_atn, answer_batch, answer_atn_batch, False ,a_tensor_train_embed)

      loss = loss_function(answer_batch, logits)
      loss.backward()
      total_loss = total_loss + loss
     
      nn.utils.clip_grad_norm_(model.parameters(),0.5)

      optimizer.step()
      
    val_loss = evaluate(model, test_loader, device, c_hid, c_tensor, c_atn, w_embed)
    print(f'Epoch {epoch}, Train_loss: {total_loss/(i+1)}, Val_loss: {val_loss}')
     
  return model


In [31]:
model = train(model, train_loader, test_loader, device, epochs,c_hid, c_tensor, c_atn, w_embed)

Epoch 0, Train_loss: 4.800919532775879, Val_loss: 6.027108669281006
Epoch 1, Train_loss: 3.3890469074249268, Val_loss: 6.017892837524414
Epoch 2, Train_loss: 3.0446395874023438, Val_loss: 5.944858551025391
Epoch 3, Train_loss: 2.972137928009033, Val_loss: 5.950615406036377
Epoch 4, Train_loss: 2.794565200805664, Val_loss: 5.969342231750488
Epoch 5, Train_loss: 2.5294294357299805, Val_loss: 6.014833450317383
Epoch 6, Train_loss: 2.282773971557617, Val_loss: 6.0149736404418945
Epoch 7, Train_loss: 1.9672931432724, Val_loss: 6.016326427459717
Epoch 8, Train_loss: 1.7333730459213257, Val_loss: 6.019829273223877
Epoch 9, Train_loss: 1.5785183906555176, Val_loss: 5.99925422668457
Epoch 10, Train_loss: 1.4628888368606567, Val_loss: 6.005346298217773
Epoch 11, Train_loss: 1.3636518716812134, Val_loss: 6.044302940368652
Epoch 12, Train_loss: 1.3272987604141235, Val_loss: 6.007230758666992
Epoch 13, Train_loss: 1.238893747329712, Val_loss: 6.025773525238037
Epoch 14, Train_loss: 1.22008001804351

In [0]:
 
 with torch.no_grad():
      check_logits = model(w_embed, q_hid_test[-48:-36], c_hid, c_tensor, c_atn, test=True)
   


In [33]:
print(tokenizer.convert_ids_to_tokens(check_logits[1][11]))


['[CLS]', 'образование', 'лока', 'участка', 'способных', '##ше', '##ше', '[unused1]', '##анию', 'оценок', '##анию', '[unused1]', '[unused1]', '[unused1]', '[unused1]', '[unused1]']


In [34]:
data.iloc[-36]

Question                           Развитие персонала - это: 
Answer      процесс подготовки сотрудника к выполнению нов...
Name: 644, dtype: object

Sentence has no sence at the moment. what can we do next - use different batch size (batch is too small), not enough data - use Russian pretrained decoder, maybe should use ideas from T5

In [0]:
del model
del check_logits


In [0]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
vocab_size = tokenizer.vocab_size

In [37]:
vocab_size

119547

In [0]:
#другая попытка -  взять предобученный на соц сетях руберт и попытаться его исп. Берт плохо подходит для генерации текста, но необходимо было проверить

In [0]:
model_gen = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

In [0]:
softm = torch.nn.Softmax(dim=-1)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
class HRbotAnswerGen(nn.Module):
  def __init__(self):
    super(HRbotAnswerGen, self).__init__()

    # self.tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

    self.Q = nn.Linear(embedding_dim, attn_size)
    torch.nn.init.xavier_uniform(self.Q.weight)
    self.K = nn.Linear(embedding_dim, attn_size)
    torch.nn.init.xavier_uniform(self.K.weight)
    self.V = nn.Linear(embedding_dim, embedding_dim)
    torch.nn.init.xavier_uniform(self.V.weight)
    self.qc = model_qc
    # self.embed = nn.Embedding(self.tokenizer.vocab_size,embedding_dim)
    

    self.model_gen = model_gen

    
  def forward(self, w_embed, q_hid_batch, c_hid, c_tensor, c_atn, a_tensor = None, a_tensor_atn = None, test = False, a_tensor_embed = None):
    
    q = 0
    for question in q_hid_batch:
      cos_q_tensor = question.repeat(c_hid.size(0),1)
      cos_qc_tensor = cos(cos_q_tensor, c_hid)
      _, attn_index = torch.sort(cos_qc_tensor, descending=True)
      attn_index = attn_index[:10]
      for index in attn_index:
        w_embed_at = w_embed[index]
        c_tensor_at = c_tensor[index]
        # c_atn_at = c_atn[index]
        if attn_index[0] == index:
          w_embed_at_total = w_embed_at.clone()
          c_tensor_at_total = c_tensor_at.clone()
          # c_atn_at_total = c_atn_at.clone()
        else:
          w_embed_at_total = torch.cat([w_embed_at_total, w_embed_at], dim=0)
          c_tensor_at_total = torch.cat([c_tensor_at_total, c_tensor_at], dim=0)
          # c_atn_at_total = torch.cat([c_atn_at_total, c_atn_at], dim=0)
      if q == 0:
        question_total = w_embed_at_total.unsqueeze(0).clone()
        question_context = c_tensor_at_total.unsqueeze(0).clone()
        # question_c_atn = c_atn_at_total.unsqueeze(0).clone()
        q = q + 1
      else:
        question_total = torch.cat([question_total, w_embed_at_total.unsqueeze(0)], dim=0)
        question_context = torch.cat([question_context, c_tensor_at_total.unsqueeze(0)], dim=0)
        # question_c_atn = torch.cat([question_c_atn, c_atn_at_total.unsqueeze(0)], dim=0)    

    q_hid_batch = q_hid_batch.unsqueeze(1)
    Q_question = self.Q(q_hid_batch)
    K_wembed = self.K(question_total)
    V_embedgen = self.V(question_total)
    softm_val = softm(torch.matmul(Q_question, K_wembed.permute(0,2,1))/(attn_size**(1/2)))
    softm_val = softm_val.squeeze(1)


    # --------for embedding input--------------------------------
    # softm_matrix = torch.zeros(question_total.size(1),question_total.size(1)).repeat(batch_size,1,1).to(device)
    # for i, _ in enumerate(softm_matrix):
    #   softm_matrix[i][torch.eye(question_total.size(1)).byte()] =softm_val[i]
    # question_context_embed = torch.matmul(softm_matrix, V_embedgen)
    


    #----------for index input----------------------------------
    softm_val = softm_val/torch.max(softm_val, dim=1).values.unsqueeze(1)
    softm_val = torch.round(softm_val)
    question_context = softm_val * question_context
   
    # self.model_gen.encoder.requires_grad_=False
    # self.model_gen.decoder.requires_grad_=False
    # self.model_gen.decoder.cls.requires_grad_=True

    
    del cos_q_tensor
    del cos_qc_tensor
    del w_embed_at_total
    del c_tensor_at_total
    # del c_atn_at_total
    del w_embed_at
    del c_tensor_at
    # del c_atn_at  
    
    # del question_c_atn
    
  
    torch.cuda.empty_cache()
   
   
    question_context_atn = (question_context != 0)*1
  
  
    output  =  self.model_gen(question_context.long(), question_context_atn.long())[0]
    output = softm(output)
    output=torch.argmax(output, dim=-1)
    output_emb, _ = self.qc(output, (output!=0)*1)
 
    return output, output_emb

In [0]:
model = HRbotAnswerGen()

In [0]:
epochs =3

In [0]:
criterion = torch.nn.CosineEmbeddingLoss()

In [0]:
def evaluate(model, test_loader, device, c_hid, c_tensor, c_atn, w_embed):
  model.eval()

  for i, (q_hid_batch_test, answer_batch_test, answer_atn_batch_test) in enumerate(test_loader):
    q_hid_batch_test, answer_batch_test, answer_atn_batch_test = q_hid_batch_test.to(device), answer_batch_test.to(device), answer_atn_batch_test.to(device)
    
    with torch.no_grad():

      test_output = model(w_embed, q_hid_batch_test, c_hid, c_tensor, c_atn, test=True)[1]
      test_output = test_output[:,0]
      answer_batch_test, _ = model_qc(answer_batch_test, answer_atn_batch_test)
      answer_batch_test = answer_batch_test[:,0]
    #use scores as logits
    
    test_loss = criterion(test_output, answer_batch_test, torch.tensor([1]*len(test_output)).cuda())
    return test_loss

In [0]:
def train(model, train_loader, test_loader, device, epochs,c_hid, c_tensor, c_atn, w_embed):
  model.to(device)
  model.train()
  c_hid, c_tensor, c_atn, w_embed = c_hid.to(device), c_tensor.to(device), c_atn.to(device), w_embed.to(device)
  optimizer = AdamW(model.parameters(), lr=learning_rate)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
  for epoch in range(epochs):
    
    total_loss = 0
    for i , (q_hid_batch, answer_batch, answer_atn_batch, a_tensor_train_embed) in enumerate(train_loader):
      optimizer.zero_grad()
      
      q_hid_batch, answer_batch, answer_atn_batch, a_tensor_train_embed = q_hid_batch.to(device), answer_batch.to(device), answer_atn_batch.to(device), a_tensor_train_embed.to(device)

      
      logits = model(w_embed, q_hid_batch, c_hid, c_tensor, c_atn, answer_batch, answer_atn_batch, False ,a_tensor_train_embed)[1]

      
      
      logits_emb = logits[:,0]
      a_tensor_train_embed = a_tensor_train_embed[:, 0]

      loss = criterion(logits_emb, a_tensor_train_embed, torch.tensor([1]*len(logits_emb)).cuda())
      loss.backward()
      total_loss = total_loss + loss
     
      nn.utils.clip_grad_norm_(model.parameters(),0.5)

      optimizer.step()
      
    val_loss = evaluate(model, test_loader, device, c_hid, c_tensor, c_atn, w_embed)
    print(f'Epoch {epoch}, Train_loss: {total_loss/(i+1)}, Val_loss: {val_loss}')
     
  return model



In [0]:
#settings
batch_size = 4
embedding_dim = q_hid_train.size()[1]
attn_size = 16
learning_rate = 0.0005

In [47]:
model = train(model, train_loader, test_loader, device, epochs,c_hid, c_tensor, c_atn, w_embed)

Epoch 0, Train_loss: 0.335764616727829, Val_loss: -8.940696716308594e-08
Epoch 1, Train_loss: 0.3121093809604645, Val_loss: -5.960464477539063e-08
Epoch 2, Train_loss: 0.310528427362442, Val_loss: -2.9802322387695312e-08
Epoch 3, Train_loss: 0.3097914457321167, Val_loss: -1.4901161193847656e-08
Epoch 4, Train_loss: 0.3093159794807434, Val_loss: 0.0


In [0]:
 
 with torch.no_grad():
      check_logits = model(w_embed, q_hid_train[-40:-36], c_hid, c_tensor, c_atn, test=True)[0]

In [35]:
print(tokenizer.convert_ids_to_tokens(check_logits[3]))

['верет', '##eng', 'остан', '##gu', '##gu', 'истины', 'истины', 'Пуч', 'уголовно', '##come', '##र', '##र', '##र', '##र', '##come', '##र', 'столько', 'Пуч', '##нуют', '##нуют', 'Пуч', '##нуют', '##oci', 'Пуч', 'уголовно', '##oci', '##come', '##र', '##र', '##र', '##र', 'столько', 'бульдоз', 'наночаст', 'наночаст', 'открыло', 'наночаст', 'наночаст', 'наночаст', 'наночаст', 'открыло', 'вят', '##eng', '##र', '##र', '##र', '##र', '##र', '##र', '##eng', 'остан', 'наночаст', 'наночаст', 'наночаст', 'открыло', 'открыло', 'открыло', 'личное', 'калмыков', '##र', '##र', '##र', '##र', '##र', '##र', '##र', '##eng', 'бульдоз', 'открыло', 'открыло', 'открыло', 'личное', 'личное', 'личное', '##र', '##र', 'объективов', 'волостей', 'волостей', 'волостей', 'волостей', 'волостей', 'волостей', 'медитации', '##gu', 'медитации', 'медитации', 'почетный', 'Обыч', 'Обыч', 'почетный', 'Пуч', '##арра', 'географическом', 'действий', 'Колчака', 'Колчака', 'волостей', '##ыша', 'телевизионной', '##иваемые', 'Шаховская

In [55]:
Question_dataset_train[0][1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [0]:

data = model_gen(Question_dataset_train[0][0].unsqueeze(0).cuda(), Question_dataset_train[0][1].unsqueeze(0).cuda())

In [64]:
print(tokenizer.convert_ids_to_tokens(torch.argmax(data[0][0], dim=-1)))

['##ожд', '##апад', '##апад', '##апад', 'Терек', 'There', 'ɲ', 'Детали', '##ga', 'tem', 'Cher', 'Предприят', 'Предприят', 'Предприят', 'Sem', '##емар', '##емар', 'Sem', '##емар', 'Терек', 'Терек', 'Терек', 'Siemens', 'лотов', '##yx', '##енья', 'Предприят', '##yx', 'литературному', 'литературному', 'оперативных', '##емар', 'становились', 'становились', 'становились', '##апад', 'Sem', 'Терек', 'Терек', 'любили', 'Siemens', 'Терек', '##yx', '##енья', '##енья', '##апад', '##апад', '##апад', '##апад', 'становились', '##апад', 'становились', 'Терек', 'Терек', 'Терек', 'Терек', 'любили', 'любили', 'Терек', 'Предприят', '##yx', '##енья', 'Дмитриевич', 'Луз', 'Комиссии', '##емар', 'становились', 'становились', '##апад', 'становились', 'Терек', 'летию', 'любили', '##ways', 'Терек', '##yx', '##енья', 'Предприят', '##yx', 'Комиссии', 'соглашениям', 'перестрелку', 'перестрелку', '##емар', '##емар', 'перестрелку', 'ŵ', 'ŵ', '##ways', 'летию', 'госбезопасности', 'Терек', 'Предприят', '##yx', 'Предпри